In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import tensorflow as tf
from sklearn.model_selection import train_test_split
import scipy.misc
image = tf.keras.preprocessing.image

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
output_layer_classes = 196

In [4]:
base_path = '/home/maxim/notebooks/ObjectRecognition/datasets'

def get_path(path):
    return os.path.join(base_path, path)

In [5]:
mat = scipy.io.loadmat(get_path('devkit/cars_meta.mat'))
label_names = np.array([row[0] for row in mat['class_names'].reshape(-1)])

In [6]:
mat = scipy.io.loadmat(get_path('devkit/cars_annos.mat'))
file_labels = {row[0][0]: row[5].squeeze() - 1 for row in mat['annotations'][0]}

filenames_np = np.array([get_path(row[0][0]) for row in mat['annotations'][0]])

labels_np = np.array([row[5].squeeze() - 1 for row in mat['annotations'][0]])
labels_one_hot = np.zeros((labels_np.size, output_layer_classes))
labels_one_hot[np.arange(labels_np.size), labels_np] = 1

In [7]:
filenames_train_np, filenames_test_np, labels_train_np, labels_test_np = \
    train_test_split(filenames_np, labels_one_hot, test_size=0.05, random_state=42)

In [8]:
filenames_train = tf.constant(filenames_train_np)
labels_train = tf.constant(labels_train_np)
filenames_test = tf.constant(filenames_test_np)
labels_test = tf.constant(labels_test_np)

In [9]:
dataset_train = tf.data.Dataset.from_tensor_slices((filenames_train, labels_train))
dataset_test = tf.data.Dataset.from_tensor_slices((filenames_test, labels_test))

image_count = int(filenames_train.shape[0])
IMG_WIDTH = 224
IMG_HEIGHT = 224
CHANNELS = 3
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.ceil(2 * image_count / BATCH_SIZE)


def convert(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
     # img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    return img, label

def augment(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
     # img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.cast(img, tf.float32)
    img = (img / 127.5) - 1
    img = tf.image.resize(img, [2 * IMG_WIDTH, 2 * IMG_HEIGHT])
    
    img = tf.image.random_crop(img, size=[IMG_WIDTH, IMG_HEIGHT, CHANNELS]) # Random crop back to 28x28
    img = tf.image.random_brightness(img, max_delta=0.5) # Random brightness
    img = tf.image.random_flip_left_right(img)

    return img,label

database_train1 = dataset_train.map(convert)
database_train2 = dataset_train.map(augment)
database_train = database_train1.concatenate(database_train2)

database_test = dataset_test.map(convert)

In [10]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(label_names[label_batch[n]])
        plt.axis('off')


In [11]:
train_ds = prepare_for_training(database_train, cache='cache')
test_ds = prepare_for_training(database_test)

In [12]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, CHANNELS)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [13]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [14]:
base_model.trainable = True
# Fine-tune from this layer onwards
# fine_tune_at = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable =  False


In [15]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(output_layer_classes)

In [16]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [17]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
initial_epochs = 10
validation_steps=20

In [19]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=test_ds,
                    validation_steps=validation_steps)


Train for 481.0 steps, validate for 20 steps
Epoch 1/10
481/481 [==============================] - 128s 266ms/step - loss: 3.8474 - accuracy: 0.2024 - val_loss: 3.6380 - val_accuracy: 0.1469
Epoch 2/10
481/481 [==============================] - 117s 242ms/step - loss: 1.7243 - accuracy: 0.6172 - val_loss: 2.4835 - val_accuracy: 0.3875
Epoch 3/10
481/481 [==============================] - 117s 242ms/step - loss: 0.9591 - accuracy: 0.7921 - val_loss: 1.8095 - val_accuracy: 0.5750
Epoch 4/10
481/481 [==============================] - 117s 243ms/step - loss: 0.5585 - accuracy: 0.8947 - val_loss: 1.4251 - val_accuracy: 0.6492
Epoch 5/10
481/481 [==============================] - 117s 243ms/step - loss: 0.3224 - accuracy: 0.9517 - val_loss: 1.1070 - val_accuracy: 0.7242
Epoch 6/10
481/481 [==============================] - 117s 244ms/step - loss: 0.1849 - accuracy: 0.9799 - val_loss: 0.9656 - val_accuracy: 0.7422
Epoch 7/10
481/481 [==============================] - 117s 243ms/step - loss: 0

In [20]:
loss0,accuracy0 = model.evaluate(test_ds, steps=validation_steps)
print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 1s 55ms/step - loss: 0.6690 - accuracy: 0.8156
loss: 0.67
accuracy: 0.82


In [21]:
model.save('cars_full_aug10_double_data.h5')

In [22]:
history1 = model.fit(train_ds,
                     epochs=1,
                     steps_per_epoch=STEPS_PER_EPOCH,
                     validation_data=test_ds,
                     validation_steps=validation_steps)

Train for 481.0 steps, validate for 20 steps
481/481 [==============================] - 117s 243ms/step - loss: 0.0276 - accuracy: 0.9975 - val_loss: 0.6349 - val_accuracy: 0.8211


In [23]:
loss0,accuracy0 = model.evaluate(test_ds, steps=validation_steps)
print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 1s 60ms/step - loss: 0.6629 - accuracy: 0.8156
loss: 0.66
accuracy: 0.82


In [24]:
history1 = model.fit(train_ds,
                     epochs=1,
                     steps_per_epoch=STEPS_PER_EPOCH,
                     validation_data=test_ds,
                     validation_steps=validation_steps)

Train for 481.0 steps, validate for 20 steps
481/481 [==============================] - 117s 243ms/step - loss: 0.0239 - accuracy: 0.9976 - val_loss: 0.6529 - val_accuracy: 0.8281


In [25]:
loss0,accuracy0 = model.evaluate(test_ds, steps=validation_steps)
print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

20/20 [==============================] - 1s 61ms/step - loss: 0.6380 - accuracy: 0.8320
loss: 0.64
accuracy: 0.83


In [26]:
model.save('cars_full_aug12_double_data.h5')